# Dimension tables 

In [3]:
import pandas as pd
from pathlib import Path
from datetime import datetime

SRC = "Project2_Dataset_Corrected.csv"
OUT = Path("csv_out")
OUT.mkdir(exist_ok=True)

# 1. Read as string so nothing gets coerced unexpectedly
df = pd.read_csv(SRC, dtype=str)

# 2. Trim whitespace only (no fill‑na, no sentinel values)
df = df.apply(lambda s: s.str.strip())

# Checking column names:
df.columns

Index(['ID', 'Crash ID', 'State', 'Month', 'Year', 'Dayweek', 'Time',
       'Crash Type', 'Number Fatalities', 'Bus Involvement',
       'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement',
       'Speed Limit', 'Road User', 'Gender', 'Age',
       'National Remoteness Areas', 'SA4 Name 2021', 'National LGA Name 2024',
       'National Road Type', 'Christmas Period', 'Easter Period', 'Age Group',
       'Day of week', 'Time of day'],
      dtype='object')

In [4]:
# --- QA: record counts & null audits ---------------------------------
orig_rows  = df.shape[0]
dup_crash  = df.duplicated('Crash ID').sum()
dup_person = df.duplicated('ID').sum()
null_summary = df.isna().sum().sort_values(ascending=False).head(8)

print(f"Rows read          : {orig_rows:,}")
print(f"Duplicate crash_id : {dup_crash:,}   ← normal, multiple victims per crash")
print(f"Duplicate person_id: {dup_person:,}")
display(null_summary)

# hard-stop the notebook only if the surrogate key is not unique
assert dup_person == 0, "`ID` column is not unique!"


Rows read          : 10,490
Duplicate crash_id : 807   ← normal, multiple victims per crash
Duplicate person_id: 0


ID                        0
Road User                 0
Day of week               0
Age Group                 0
Easter Period             0
Christmas Period          0
National Road Type        0
National LGA Name 2024    0
dtype: int64

# Data-quality check:

| Metric | Value | Comment |
|--------|-------|---------|
| **Rows read** | {{orig_rows:,}} | all rows present |
| **Duplicate `Crash ID`** | {{dup_crash:,}} | expected – same crash, many people |
| **Duplicate `ID`** | {{dup_person:,}} | must be 0 |
| **Top-8 null columns** | see below | all zero |


In [52]:
# Checking data types:
df.dtypes

ID                               object
Crash ID                         object
State                            object
Month                            object
Year                             object
Dayweek                          object
Time                             object
Crash Type                       object
Number Fatalities                object
Bus Involvement                  object
Heavy Rigid Truck Involvement    object
Articulated Truck Involvement    object
Speed Limit                      object
Road User                        object
Gender                           object
Age                              object
National Remoteness Areas        object
SA4 Name 2021                    object
National LGA Name 2024           object
National Road Type               object
Christmas Period                 object
Easter Period                    object
Age Group                        object
Day of week                      object
Time of day                      object


In [53]:
import pandas as pd

import pandas as pd

def describe_categorical_columns(df: pd.DataFrame):
    """
    For each column with dtype 'object' or 'category', print:
      • column name and dtype
      • number of unique values
      • list of unique values
    """
    # pick object‐ and category‐typed columns
    cat_cols = df.select_dtypes(include=['object','category']).columns

    if len(cat_cols) == 0:
        print("No categorical columns found.")
        return

    for col in cat_cols:
        ser = df[col]
        uniques = ser.dropna().unique().tolist()
        print(f"{col!r} ({ser.dtype}):")
        print(f"  • # unique values : {len(uniques)}")
        print(f"  • values          : {uniques}\n")

describe_categorical_columns(df)

'ID' (object):
  • # unique values : 10490
  • values          : ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149',

In [54]:
# 3. Ensure time always HH:MM:SS  (00:00 stays 00:00)
df["Time"] = (
    pd.to_datetime(df["Time"], format="%H:%M", errors="coerce")
      .dt.strftime("%H:%M:%S")
      .fillna("00:00:00")
)

# 4. Rename columns to snake_case (unchanged list)
df = df.rename(columns={
    "ID": "person_id", "Crash ID": "crash_id",
    "State":"state_name", "SA4 Name 2021":"sa4_name",
    "National LGA Name 2024":"lga_name",
    "National Remoteness Areas":"remoteness_name",
    "National Road Type":"road_type_name",
    "Day of week":"day_flag", "Time of day":"time_of_day",
    "Dayweek":"dayweek_text", "Time":"time_text",
    "Crash Type":"crash_type", "Number Fatalities":"num_fatalities",
    "Speed Limit":"speed_limit", "Road User":"road_user",
    "Gender":"gender", "Age":"age", "Age Group":"age_group"
})

# 5. Cast the genuinely numeric columns (no missing → no error)
for col in ["Month","Year","num_fatalities","speed_limit","age"]:
    df[col] = df[col].astype(int)

In [55]:
# Dimension tables:
def write_dim(col): (
    df[[col]].drop_duplicates().sort_values(col)
      .to_csv(OUT/f"{col}.csv", index=False))

for col in ["state_name","sa4_name","lga_name","remoteness_name",
            "road_type_name","day_flag","time_of_day","dayweek_text"]:
    write_dim(col)

pd.DataFrame({"period_name":["Christmas Period","Easter Period"]}) \
  .to_csv(OUT/"holiday_period.csv", index=False)

# 7. Vehicle bridge  — case exactly “Yes”/“No”
veh = []
for src, vtype in [("Bus Involvement","bus"),
                   ("Heavy Rigid Truck Involvement","heavyrigidtruck"),
                   ("Articulated Truck Involvement","articulatedtruck")]:
    tmp = df.loc[df[src]=="Yes", ["crash_id"]].copy()
    tmp["vehicle_type"] = vtype
    veh.append(tmp)
pd.concat(veh).to_csv(OUT/"vehicle.csv", index=False)

# 8. Crash fact
crash_cols = [
    "crash_id","Month","Year","dayweek_text","time_text","crash_type",
    "num_fatalities","speed_limit",
    "state_name","sa4_name","lga_name",
    "remoteness_name","road_type_name",
    "Christmas Period","Easter Period","day_flag","time_of_day"
]
(df[crash_cols]
   .drop_duplicates("crash_id")
   .rename(columns={"Month":"month","Year":"year",
                    "Christmas Period":"christmas_period",
                    "Easter Period":"easter_period"})
   .assign(load_ts=datetime.utcnow().isoformat(timespec="seconds"))
   .to_csv(OUT/"crash.csv", index=False))

# 9. Person fact
(df[["person_id","crash_id","road_user","gender","age","age_group"]]
   .assign(load_ts=datetime.utcnow().isoformat(timespec="seconds"))
   .to_csv(OUT/"person.csv", index=False))

print("Clean export complete – no sentinel values, no data lost.")

Clean export complete – no sentinel values, no data lost.
